In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.autograd import Variable
import torchvision
import os
import random

In [ ]:
import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import matplotlib.pyplot as plt # data visualization library
import sklearn
from sklearn.datasets import fetch_olivetti_faces

# Exercise : Convolution Neural Network

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [ ]:
# Download data and define the transformer (data normalization)
# Do not change this function
def load_data(seed_value=42):
    MNIST_normalize = transforms.Normalize((0.1307,), (0.3081,))
    MNIST_transform = transforms.Compose([
      transforms.ToTensor(),
      MNIST_normalize
    ])

    # MNIST Dataset
    train = datasets.MNIST(root='./data/',
                                train=True,
                                transform=MNIST_transform,
                                download=True)

    test = datasets.MNIST(root='./data/',
                                train=False,
                                transform=MNIST_transform)

    #print(valid.dataset.data.shape)
    train_subset_size = 5000
    train.data = train.data[0:train_subset_size]
    train.targets = train.targets[0:train_subset_size]

    train_size = int(0.9 * train_subset_size)
    val_size =  int(0.1 * train_subset_size)
    print("train_set_size:", train_size)
    print("val_set_size:", val_size)

    train, valid = random_split(train, [train_size, val_size], generator=torch.Generator().manual_seed(seed_value))

    return train, valid, test

In [ ]:
# Function to show data
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# Data Loader 
def get_data_loaders(dataset, batch_size, shuffle=True):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                            batch_size=batch_size,
                                            shuffle=shuffle)
    return loader

In [ ]:
# Model
class Conv_Net(nn.Module):
    def __init__(self, dropout_value=0.5):
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, kernel_size=3)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3)
        self.mp = nn.MaxPool2d(2)
        self.drop = nn.Dropout(dropout_value)
        self.fc = nn.Linear(6400, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.drop(x)
        x = self.fc(x)
        return F.log_softmax(x)

In [ ]:
# Run one training epoch
def train(epoch, train_loader, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).to(device), Variable(target).to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))


In [ ]:
# Compute metrics
def test(test_loader, device, is_train_set=True):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data).to(device), Variable(target).to(device)
            output = model(data)
            # sum up batch loss
            test_loss += F.nll_loss(output, target, size_average=False).data
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\n{}: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        'Train evaluation' if is_train_set else 'Val/Test evaluation', test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Execution with default hyperparameters (A & B)

In [ ]:
# Use the default seed
seed_everything(seed=42)

# Training settings
batch_size = 128
lr = 0.1
momentum = 0.9 
l2 = 0.0
dropout_value = 0.0
num_epochs = 15

# Select the device
# device = 'cuda'
device = 'cpu'

# Define the network
model = Conv_Net(dropout_value).to(device)

# Define the optimizer 
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay= l2)

train_dataset, val_dataset, test_dataset = load_data()

# Divide the available data into training and validation datasets
train_loader = get_data_loaders(train_dataset, batch_size, shuffle=True)
val_loader = get_data_loaders(val_dataset, batch_size, shuffle=False)
test_loader = get_data_loaders(test_dataset, batch_size, shuffle=False)

# Run the training epochs
for epoch in range(1, num_epochs):
    train(epoch, train_loader, device)
    test(train_loader, device, is_train_set=True)
    test(val_loader, device, is_train_set=False)

# Compute the metrics using the testing dataset
test(test_loader, device, is_train_set=False)

# Execution for C

Notes: Increasing dropout to 0.5 provides a better result than the default hyperparams. Accuracy is 1% higher, and the loss goes from 0.27 to 0.08. It likely generalizes better with this change and therefore prevents overfitting of the test data. It is also MASSIVELY ACCURATE on the training data with an accuracy of 99%.

In [ ]:
# Use the default seed
seed_everything(seed=42)

# Training settings
batch_size = 128
lr = 0.1
momentum = 0.9 
l2 = 0.0
dropout_value = 0.5
num_epochs = 15

# Select the device
# device = 'cuda'
device = 'cpu'

# Define the network
model = Conv_Net(dropout_value).to(device)

# Define the optimizer 
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay= l2)

train_dataset, val_dataset, test_dataset = load_data()

# Divide the available data into training and validation datasets
train_loader = get_data_loaders(train_dataset, batch_size, shuffle=True)
val_loader = get_data_loaders(val_dataset, batch_size, shuffle=False)
test_loader = get_data_loaders(test_dataset, batch_size, shuffle=False)

# Run the training epochs
for epoch in range(1, num_epochs):
    train(epoch, train_loader, device)
    test(train_loader, device, is_train_set=True)
    test(val_loader, device, is_train_set=False)

# Compute the metrics using the testing dataset
test(test_loader, device, is_train_set=False)

# Execution for D

Notes: The increase of l2 from 0.00 to 0.05 dropped the accuracy from 97% to 87%. The train accuracy is higher than the prediction accuracy at 88%, indicating that the model might be overfitting.

In [ ]:
# Use the default seed
seed_everything(seed=42)

# Training settings
batch_size = 128
lr = 0.1
momentum = 0.9 
l2 = 0.05
dropout_value = 0.0
num_epochs = 15

# Select the device
# device = 'cuda'
device = 'cpu'

# Define the network
model = Conv_Net(dropout_value).to(device)

# Define the optimizer 
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay= l2)

train_dataset, val_dataset, test_dataset = load_data()

# Divide the available data into training and validation datasets
train_loader = get_data_loaders(train_dataset, batch_size, shuffle=True)
val_loader = get_data_loaders(val_dataset, batch_size, shuffle=False)
test_loader = get_data_loaders(test_dataset, batch_size, shuffle=False)

# Run the training epochs
for epoch in range(1, num_epochs):
    train(epoch, train_loader, device)
    test(train_loader, device, is_train_set=True)
    test(val_loader, device, is_train_set=False)

# Compute the metrics using the testing dataset
test(test_loader, device, is_train_set=False)

# Exercise : MF Faces

In [ ]:
faces = fetch_olivetti_faces()
print(faces.DESCR)

Definition of the data matrix:

In [ ]:
D = faces.data

The observations/rows pf the data matrix can be visualized as pictures. 10 consecutive pictures are taken from one person. This are the first 30 pictutres.

In [ ]:
# Here are the first ten guys of the dataset
fig = plt.figure(figsize=(10, 3))
for i in range(30):
    ax = plt.subplot2grid((3, 10), (int(i/10), i-int(i/10)*10))
    
    ax.imshow(D[i,:].reshape(64, 64), cmap=plt.cm.gray)
    ax.axis('off')

# Exercise: k-means Initialization

In [19]:
!pip3 show scikit-learn

Name: scikit-learn
Version: 1.2.0
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\20200876\appdata\local\packages\pythonsoftwarefoundation.python.3.9_qbz5n2kfra8p0\localcache\local-packages\python39\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 


In [20]:
!pip3 show numpy

Name: numpy
Version: 1.24.1
Summary: Fundamental package for array computing in Python
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD-3-Clause
Location: c:\users\20200876\appdata\local\packages\pythonsoftwarefoundation.python.3.9_qbz5n2kfra8p0\localcache\local-packages\python39\site-packages
Requires: 
Required-by: bokeh, matplotlib, pandas, scikit-learn, scipy, seaborn


If your versions don't match, the following commands (or their anaconda version) could help to get the newest stable release. If you need help with this, please ask the TAs during instruction hours.

In [21]:
!pip3 install scikit-learn --upgrade
!pip3 install numpy --upgrade

     --------------------------------------- 14.7/14.7 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24.1:
      Successfully uninstalled numpy-1.24.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\20200876\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


  Using cached numpy-1.24.1-cp39-cp39-win_amd64.whl (14.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.6.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.24.1 which is incompatible.
You should consider upgrading via the 'C:\Users\20200876\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


The functions generating the datasets are given here:

In [23]:
def generateMoons(epsilon, n):
    moons, labels = sklearn.datasets.make_moons(n_samples=n, noise=epsilon, random_state=7)
    return "moons", moons, labels, 2
def generateBlobs(epsilon, n):
    blobs, labels = sklearn.datasets.make_blobs(n_samples=n,centers=3, cluster_std=[epsilon + 1, epsilon + 2.5, epsilon + 0.5])
    return "blobs", blobs, labels, 3

Extra function needed

In [24]:
def computeP(D,X):
    n = D.shape[0]
    P = np.zeros(n)
    denominator = np.sum([np.linalg.norm(D[j,:].T - X) for j in range(n)])
    for i in range(n):
        P[i] = np.linalg.norm(D[i,:].T - X) / denominator
    return P

In [25]:
def computeI(D,X,l,i_list):
    dist_list=[]
    for i in i_list:
        minus_term = np.concatenate(X,D[i,:].T)
        sum = np.sum([np.linalg.norm(D[j,:].T - minus_term) for j in range(n)])
        dist_list.append(sum)
    return 0

Implement the centroid initialization here. Right now, it returns a random initialization. 

In [26]:
def init_centroids_greedy_pp(D,r,l=10):
    '''
        :param r: (int) number of centroids (clusters)
        :param D: (np-array) the data matrix
        :param l: (int) number of centroid candidates in each step
        :return: (np-array) 'X' the selected centroids from the dataset
    '''   
    rng =  np.random.default_rng(seed=7) # use this random generator to sample the candidates (sampling according to given probabilities can be done via rng.choice(..))
    n,d = D.shape

    indexes = rng.integers(low=0, high=n, size=r)
    X = np.array(D[indexes,:]).T
    s = 2
    while s <= r:
        P = computeP(D,X)
        i_list = np.random.choice(n, size=l, p=P, replace=False)
        i = np.argmin(computeI)
        X = np.hstack((X, D[i, :].reshape(d, 1)))
        s += 1
    return X

In [18]:
init_centroids_greedy_pp(D, r = 2)

NameError: name 'D' is not defined

In [6]:
import scipy
def spectral_clustering(W,r, X_init):
    '''
        :param W: (np-array) nxn similarity/weighted adjacency matrix
        :param r: (int) number of centroids (clusters)
        :param X_init: (function) the centroid initialization function 
        :return: (np-array) 'Y' the computed cluster assignment matrix
    '''
    L = np.diag(np.array(W.sum(0))[0]) - W
    Lambda, V = scipy.sparse.linalg.eigsh(L, k=r+1, which="SM")
    A = V[:,1:]
    initial_points = X_init(A,r)
    X, Y = kmeans(A, r, initial_points)
    return Y

This is the $k$-means implementation from the lecture accompanying notebook.

In [7]:
def RSS(D,X,Y):
    return np.sum((D- Y@X.T)**2)

In [8]:
def getY(labels):
    '''
        Compute the cluster assignment matrix Y from the categorically encoded labels
    '''
    Y = np.eye(max(labels)+1)[labels]
    return Y
def update_centroid(D,Y):
    cluster_sizes = np.diag(Y.T@Y).copy()
    cluster_sizes[cluster_sizes==0]=1
    return D.T@Y/cluster_sizes
def update_assignment(D,X):
    dist = np.sum((np.expand_dims(D,2) - X)**2,1)
    labels = np.argmin(dist,1)
    return getY(labels)
def kmeans(D,r, X_init, epsilon=0.00001, t_max=10000):
    X = X_init.copy()
    Y = update_assignment(D,X)
    rss_old = RSS(D,X,Y) +2*epsilon
    t=0
    #Looping as long as difference of objective function values is larger than epsilon
    while rss_old - RSS(D,X,Y) > epsilon and t < t_max-1:
        rss_old = RSS(D,X,Y)
        X = update_centroid(D,Y)
        Y = update_assignment(D,X)
        t+=1
    print(t,"iterations")
    return X,Y

We generate a dataset.

In [9]:
n=500
dataID, D, labels, r = generateBlobs(0.05,n)

NameError: name 'generateBlobs' is not defined

Run kmeans and spectral clustering based on the initialization technique.

In [10]:
X_init = init_centroids_greedy_pp(D,r)
X,Y = kmeans(D,r, X_init)

NameError: name 'D' is not defined

Plot the clustering. The initial centroids are marked in red, and the final centroids are marked in blue. You can use this visualization to see if your initialization makes sense. It doesn't work for spectral clustering.

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.axis('equal')
ax.scatter(D[:, 0], D[:, 1], c=np.argmax(Y,axis=1), s=10)
ax.scatter(X_init.T[:, 0], X_init.T[:, 1], c='red', s=50, marker = 'D')
ax.scatter(X.T[:, 0], X.T[:, 1], c='blue', s=50, marker = 'D')

We generate the moons dataset and compute spectral clustering with the implemented initialization technique.

In [13]:
dataID, D, labels, r = generateMoons(0.05,n)

NameError: name 'generateMoons' is not defined

In [12]:
from sklearn.neighbors import radius_neighbors_graph, kneighbors_graph
# Implement here the computation of W as knn graph
W = radius_neighbors_graph(D,0.5,include_self=False)
Y = spectral_clustering(W,r,init_centroids_greedy_pp)

plt.scatter(D[:, 0], D[:, 1], c=np.argmax(Y,axis=1), s=10)
plt.title('%s'  % ( dataID) )
plt.show()

NameError: name 'D' is not defined

In [11]:
from sklearn.neighbors import radius_neighbors_graph, kneighbors_grap
X,Y = kmeans(D,r, X_init)
N = kneighbors_graph(D, n_neighbors=20,include_self=False, n_jobs=-1)
W = 0.5*(N+N.T)
spectral_clustering(W,r,X_init)

ImportError: cannot import name 'kneighbors_grap' from 'sklearn.neighbors' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\__init__.py)

### Therefore, the answer for 3-a is D

# Exercise : Movielens
To read the dataset you might need to alter the path to look for it:

In [29]:
# lets explore movies.csv
import pandas as pd
movies= pd.read_csv('ml-latest-small/movies.csv')
movies.head()

FileNotFoundError: [Errno 2] File ml-latest-small/movies.csv does not exist: 'ml-latest-small/movies.csv'

In [30]:
# lets explore ratings.CSV
ratings=pd.read_csv('ml-latest-small/ratings.csv',sep=',')
ratings.head()

FileNotFoundError: [Errno 2] File ml-latest-small/ratings.csv does not exist: 'ml-latest-small/ratings.csv'

The original ratings are in the range of 0.5 and 5:

In [ ]:
min(ratings["rating"]), max(ratings["rating"])

We convert the sparse representation of movie ratings into a data matrix. The missing values are filled with zeros.

In [ ]:
df_movie_ratings = ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)  #fill unobserved entries with μ
df_movie_ratings.head()

We consider here only the movies which have been rated by more than 200 users. That are 18 movies. We will not be able to infer a pattern for movies with very few observations anyways, but for this exercise we are mostly interested in the prnciple and do not need a big dataset.

In [ ]:
np.sum(np.sum(df_movie_ratings!=0,0)>200)

In [ ]:
keep_movie = np.sum(df_movie_ratings!=0,0)>200
df_D = df_movie_ratings.loc[:,keep_movie]
df_D.head()

Furthermore, we will throw out all the users which have not rated more than five movies.

In [ ]:
np.sum(np.sum(df_D!=0,1)>5)

The resulting dataset has the userID as rows and movieIDs as columns. Hence, userID 1 and 4 addresses the first two rows of this dataset.

In [ ]:
keep_user = np.sum(df_D!=0,1)>5
df_D = df_D.loc[keep_user,:]
df_D.head()

The movie number- title assignments are given as follows:

In [ ]:
movies.loc[movies['movieId'].isin(df_D.columns)]

The resulting data matrix is given as follows:

In [ ]:
D = df_D.to_numpy()
D.shape

In [ ]:
def indNonzero(D,n,m):
    O = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            if D[i][j] != 0:
                O[i][j] = 1
    return O

In [ ]:
def initalizeMatrix(n,m,r):
    X = np.random.rand(m, r)
    Y = np.random.rand(n, r)
    return X,Y

In [ ]:
def computeXk(D,k,O_xk,X,Y,l,d,r):
    D_kT = D[:,k].T
    inverse = np.linalg.inv(Y.T@O_xk@Y+l*np.eye(r))
    X[k,:] = D_kT@Y@inverse
    return X

In [ ]:
def computeYi(D,i,O_yi,X,Y,l,n,r):
    D_iT = D[i,:].T
    inverse = np.linalg.inv(X.T@O_yi@X+l*np.eye(r))
    Y[i,:] = D_iT@X@inverse
    return Y

Solution for Question A

In [ ]:
mseo = []
def completionOfMatrix(D,r,t_max,l):
    early_epoch = 1
    n = D.shape[0]
    d = D.shape[1]
    relative_change_threshold = 0.0001
    relative_change = 1
    X,Y = initializeMatrix(n,d,r) 
    O = indNonzero(D,n,d)
    card_O = O.shape[0]
    t = 1
    while t < t_max:
        for k in range(d):
            O_xk = np.diagflat(O[:,k])
            X = computeXk(D,k,O_xk,X,Y,l,d,r)
            for i in range(n):
                O_yi = np.diagflat(O[i,:])
                Y = computeYi(D,i,O_yi,X,Y,l,n,r)
        MSEO = 1/card_O * np.linalg.norm(D - O*(Y@X.T))**2
        mseo.append(MSEO)
        if t > 1:
            relative_change = abs(MSEO - mseo[-2])
        if (relative_change>relative_change_threshold) and early_epoch == 1:
            early_epoch = t
        t += 1
    return X,Y,early_epoch

X,Y,early_epoch = completionOfMatrix(D,r=5,t_max=100,l = 0.0001)

### Answer: Plot MESO

Solution for Question B

In [ ]:
x_axis_labels = [i for i in range(1, len(mseo)+1)]
plt.plot(x_axis_labels, mseo)
plt.xlabel('Epoch')
plt.ylabel('MSEO')
plt.title('MSEO-Epoch graph')

### Answer: Threshold that is early stopped, change vales to observe which one

Solution for Question D

In [31]:
selected_batch = pd.DataFrame(D[:3])
selected_batch

NameError: name 'D' is not defined

### Answer: Approximation for multiple, different lambda types

In [ ]:
selected_batch = pd.DataFrame(D[:3])
selected_batch

In [ ]:
def approximator(df):
    df = np.round(df, 0)
    return pd.DataFrame(df)

In [ ]:
lambdas = [1, 0.5, 0.1, 0.0001]
results = {}
for i in lambdas:
    X,Y,early_epoch = completionOfMatrix(D,r=5,t_max=100,l = i)
    matrix = pd.DataFrame(Y[:3]@X.T)
    print(approximator(matrix))

# Notes

### Question 1

Options:
- A. Using the default hyper-parameters, the model generalizes well since the accuracy on the test set is higher than the accuracy on the validation set.
- B. Using the default hyper-parameters, the model suffers from underfitting.
- C. Using a dropout of 0.5 reduces overfitting.
- D. Using L2 regularization of 0.05 reduces underfitting.

Elimination:
- B. Just no. The standard tuning results in 100% train accuracy and 96% test accuracy. There's definitely no underfitting.
- A. Fun fact, the accuracy on the test set is actually lower than the accuracy on the validation set if you calculate them. Contradiction.
- D. Cannot be. The only thing L2 regularization reduces is fitting. The accuracy drops massively compared to other tunings. Underfitting is a scenario where a model doesn't capture the relationship. L2 only seems to inspire underfitting.

C seems likely. The normal tuning is 100% accurate on the training set while the dropout version isn't. However, it's test accuracy is slightly improved. Thus, sure. It seems to generalize better.